# ShorelineMonitor Shorelines

The ShorelineMonitor dataset provides [Satellite-Derived Shorelines (SDS)](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/shorelinemonitor-shorelines/collection.json) extracted from annually
composited Landsat satellite imagery spanning the years 1984-2024. These shorelines offer a global
view of coastal change and shoreline dynamics, serving as a critical foundation for global coastal
monitoring and analytics.

The shorelines are LineStrings with additional attributes such as measures of curvature (sinuosity, fractal dimension and self-intersection density) as well as composite
image identifiers, cloud-coverage metrics, and some parameters used in the shoreline extraction process.

This dataset builds on earlier efforts by Luijendijk et al. (2018) and Deltares RPC, and has now been processed as cloud-native dataset (Calkoen et al. 2025) to enhance global-scale coastal analytics. The dataset is available upon reasonable request. Please contact the data provider for more information or collaboration opportunities.

In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}


coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("shorelinemonitor-shorelines")

In [ ]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

In [ ]:
snapshot.explore()

## Select a region of interest

Wait for map to render and zoom to area of interest. 

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.796, 53.108, 5.229, 53.272))
west, south, east, north = list(roi.total_bounds)

## Fetch data

In [ ]:
import coastpy

db = coastpy.io.STACQueryEngine(
    stac_collection=collection,
    storage_backend="azure",
    # columns = ["geometry", "shoreline_id", "datetime", "bbox", ] ... # when you don't need all data
)

In [ ]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
print(f"Shape: {df.shape}")
df.head()

## Plot the data on a map

Currently the hvplot() method for linestrings (paths) is broken, so we use matplotlib. 

In [ ]:
df = gpd.overlay(df, roi)
df = df.assign(year=pd.to_datetime(df.datetime).dt.strftime("%Y"))
df["year"] = df["year"].astype("int16")

In [ ]:
import colorcet as cc
import contextily as ctx
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

cmap = cc.m_bmy

df.plot(
    ax=ax,
    column="year",  #
    cmap=cmap,
    legend=True,
    legend_kwds={"label": "Year", "orientation": "vertical"},
    linewidth=1.5,
    alpha=0.8,
)


ctx.add_basemap(
    ax, crs=df.crs.to_string(), source=ctx.providers.Esri.WorldImagery
)

plt.tight_layout()
plt.show()

## Work with the full dataset

In [ ]:
import dask_geopandas

fs = fsspec.filesystem("az", **storage_options)
urlpaths = snapshot.href.to_list()
ddf = dask_geopandas.read_parquet(urlpaths, filesystem=fs)

In [ ]:
ddf